In [4]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

training_data = datasets.FashionMNIST(
    root = "data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root = "data",
    train = False,
    download = True,
    transform=ToTensor(),
)

100%|██████████| 26.4M/26.4M [00:01<00:00, 14.7MB/s]


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 29.5k/29.5k [00:00<00:00, 299kB/s]


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 4.42M/4.42M [00:00<00:00, 5.22MB/s]


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 5.15k/5.15k [00:00<00:00, 8.91MB/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [6]:
batch_size = 64
train_dataloader = DataLoader(
    dataset=training_data, 
    batch_size=batch_size, 
)
test_dataloader = DataLoader(
    dataset=training_data,
    batch_size=batch_size
)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break


Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [14]:
device = "cpu"

class NeuralNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(784, 512), 
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )
    
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
    
model = NeuralNet().to(device)
print(model)

NeuralNet(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [20]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr= 1e-3)

def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X,y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute the Prediction Error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f'loss: {loss:>7f} [{current:>5d}/ {size:>5d}]')

def test(dataloader, model, loss_fn, optimizer):
    size =len(dataloader.dataset)
    model.eval()
    num_batches = len(dataloader)
    test_loss, correct = 0,0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss/=num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100 * correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")




In [21]:
epochs  = 5
for t in range(epochs):
    print(f'Epoch {t+1} ------------------------------------')
    train(train_dataloader, model=model, loss_fn=loss_fn, optimizer=optimizer)
    test(test_dataloader, model=model, loss_fn=loss_fn, optimizer=optimizer)
print('Completed')

Epoch 1 ------------------------------------
loss: 0.721110 [   64/ 60000]
loss: 0.814084 [ 6464/ 60000]
loss: 0.591320 [12864/ 60000]
loss: 0.804614 [19264/ 60000]
loss: 0.690688 [25664/ 60000]
loss: 0.694695 [32064/ 60000]
loss: 0.771320 [38464/ 60000]
loss: 0.757540 [44864/ 60000]
loss: 0.761729 [51264/ 60000]
loss: 0.730944 [57664/ 60000]
Test Error: 
 Accuracy: 74.4%, Avg loss: 0.712224 

Epoch 2 ------------------------------------
loss: 0.691720 [   64/ 60000]
loss: 0.789226 [ 6464/ 60000]
loss: 0.567856 [12864/ 60000]
loss: 0.786688 [19264/ 60000]
loss: 0.673716 [25664/ 60000]
loss: 0.677609 [32064/ 60000]
loss: 0.749717 [38464/ 60000]
loss: 0.743256 [44864/ 60000]
loss: 0.744924 [51264/ 60000]
loss: 0.713754 [57664/ 60000]
Test Error: 
 Accuracy: 75.2%, Avg loss: 0.694203 

Epoch 3 ------------------------------------
loss: 0.665073 [   64/ 60000]
loss: 0.766165 [ 6464/ 60000]
loss: 0.547012 [12864/ 60000]
loss: 0.770585 [19264/ 60000]
loss: 0.658787 [25664/ 60000]
loss: 0.662

In [22]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0] , test_data[0][1]
with torch.no_grad():
    x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')


Predicted: "Ankle boot", Actual: "Ankle boot"
